In [1]:
import random

# Particle class for PSO
class Particle:
    def __init__(self, num_classes, num_timeslots, num_rooms):
        self.num_classes = num_classes
        self.num_timeslots = num_timeslots
        self.num_rooms = num_rooms

        self.position = []
        self.velocity = []
        for _ in range(num_classes):
            # Initialize timeslot and room with random floats within valid ranges
            self.position.append(random.uniform(0, num_timeslots - 1))  # timeslot
            self.position.append(random.uniform(0, num_rooms - 1))      # room
            self.velocity.append(0.0)
            self.velocity.append(0.0)

        self.best_position = self.position[:]
        self.best_fitness = float('inf')

    def update_velocity(self, global_best_position, w, c1, c2):
        for i in range(len(self.position)):
            r1 = random.random()
            r2 = random.random()
            cognitive = c1 * r1 * (self.best_position[i] - self.position[i])
            social = c2 * r2 * (global_best_position[i] - self.position[i])
            self.velocity[i] = w * self.velocity[i] + cognitive + social

    def update_position(self):
        for i in range(len(self.position)):
            self.position[i] += self.velocity[i]
            # Clamp position within valid bounds
            if i % 2 == 0:  # timeslot
                if self.position[i] < 0:
                    self.position[i] = 0
                elif self.position[i] > self.num_timeslots - 1:
                    self.position[i] = self.num_timeslots - 1
            else:  # room
                if self.position[i] < 0:
                    self.position[i] = 0
                elif self.position[i] > self.num_rooms - 1:
                    self.position[i] = self.num_rooms - 1

def fitness_function(position, num_classes):
    # Count scheduling conflicts: same timeslot and same room assigned to multiple classes
    schedule = dict()  # key: (timeslot, room), value: list of class indices
    conflicts = 0

    for i in range(num_classes):
        timeslot = int(round(position[2 * i]))
        room = int(round(position[2 * i + 1]))
        if (timeslot, room) not in schedule:
            schedule[(timeslot, room)] = []
        schedule[(timeslot, room)].append(i)

    for key, classes in schedule.items():
        if len(classes) > 1:
            conflicts += len(classes) - 1

    return conflicts

def pso_class_timetable(num_classes, num_timeslots, num_rooms, swarm_size=30, max_iter=100, w=0.5, c1=1.5, c2=1.5):
    swarm = [Particle(num_classes, num_timeslots, num_rooms) for _ in range(swarm_size)]

    global_best_position = swarm[0].position[:]
    global_best_fitness = float('inf')

    # Initialize global best
    for particle in swarm:
        fitness = fitness_function(particle.position, num_classes)
        particle.best_fitness = fitness
        if fitness < global_best_fitness:
            global_best_fitness = fitness
            global_best_position = particle.position[:]

    for iteration in range(max_iter):
        for particle in swarm:
            particle.update_velocity(global_best_position, w, c1, c2)
            particle.update_position()

            fitness = fitness_function(particle.position, num_classes)

            if fitness < particle.best_fitness:
                particle.best_fitness = fitness
                particle.best_position = particle.position[:]

            if fitness < global_best_fitness:
                global_best_fitness = fitness
                global_best_position = particle.position[:]

        print(f"Iteration {iteration + 1}: Best Fitness (Conflicts) = {global_best_fitness}")

        if global_best_fitness == 0:
            print("Conflict-free schedule found!")
            break

    # Print the schedule
    print("\nBest Schedule (Class -> Timeslot, Room):")
    for i in range(num_classes):
        ts = int(round(global_best_position[2 * i]))
        rm = int(round(global_best_position[2 * i + 1]))
        print(f"Class {i + 1}: Timeslot {ts}, Room {rm}")

    print(f"Total conflicts: {global_best_fitness}")

def main():
    print("Enter the details for timetable scheduling:")
    num_classes = int(input("Number of classes: "))
    num_timeslots = int(input("Number of timeslots: "))
    num_rooms = int(input("Number of rooms: "))

    pso_class_timetable(num_classes, num_timeslots, num_rooms)

if __name__ == "__main__":
    main()


Enter the details for timetable scheduling:
Number of classes: 10
Number of timeslots: 5
Number of rooms: 3
Iteration 1: Best Fitness (Conflicts) = 1
Iteration 2: Best Fitness (Conflicts) = 0
Conflict-free schedule found!

Best Schedule (Class -> Timeslot, Room):
Class 1: Timeslot 0, Room 1
Class 2: Timeslot 3, Room 2
Class 3: Timeslot 1, Room 2
Class 4: Timeslot 3, Room 1
Class 5: Timeslot 1, Room 0
Class 6: Timeslot 4, Room 0
Class 7: Timeslot 3, Room 0
Class 8: Timeslot 2, Room 1
Class 9: Timeslot 0, Room 0
Class 10: Timeslot 2, Room 2
Total conflicts: 0


In [3]:
import random

# ==============================
# Problem Setup
# ==============================
days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
slots_per_day = 6   # 6 slots per day
total_slots = len(days) * slots_per_day  # 30 slots per week

sections = [f"S{i+1}" for i in range(22)]
faculties = [chr(65+i) for i in range(11)]  # Faculty A-K

# Subjects with credits (hours per week)
subjects = {
    "CN": 4,   # 4-credit
    "AI": 4,   # 4-credit
    "PE": 3,   # 3-credit
    "OOM": 3,  # 3-credit
    "DEV": 3,  # 3-credit
    "EVS": 1,  # 1-credit
    "CC": 1,   # 1-credit
    "BIS": 1,  # 1-credit
    "MiniProject": 2  # 2-credit
}

# Assign faculties to subjects (cyclic for fairness)
subject_faculty_map = {}
i = 0
for sub in subjects:
    subject_faculty_map[sub] = faculties[i % len(faculties)]
    i += 1


# ==============================
# Helper Functions
# ==============================
def random_timetable():
    """Generate a random timetable for all sections."""
    timetable = {}
    for sec in sections:
        timetable[sec] = []
        for sub, hours in subjects.items():
            chosen_slots = random.sample(range(total_slots), hours)
            for slot in chosen_slots:
                timetable[sec].append((sub, slot, subject_faculty_map[sub]))
    return timetable


def fitness(timetable):
    """Calculate conflicts: minimize this."""
    conflicts = 0
    faculty_schedule = {fac: set() for fac in faculties}

    for sec in sections:
        sec_slots = {}
        for sub, slot, fac in timetable[sec]:
            # Check section conflict
            if slot in sec_slots:
                conflicts += 1
            else:
                sec_slots[slot] = sub

            # Check faculty conflict
            if slot in faculty_schedule[fac]:
                conflicts += 1
            else:
                faculty_schedule[fac].add(slot)
    return conflicts


def repair(timetable):
    """Fix timetable conflicts by reassigning slots."""
    used_slots = {sec: set() for sec in sections}
    faculty_slots = {fac: set() for fac in faculties}

    for sec in sections:
        new_entries = []
        for sub, slot, fac in timetable[sec]:
            # If conflict, pick a new free slot
            tries = 0
            while (slot in used_slots[sec]) or (slot in faculty_slots[fac]):
                slot = random.randint(0, total_slots - 1)
                tries += 1
                if tries > 100:  # emergency break
                    break
            new_entries.append((sub, slot, fac))
            used_slots[sec].add(slot)
            faculty_slots[fac].add(slot)
        timetable[sec] = new_entries
    return timetable


# ==============================
# PSO Implementation
# ==============================
class Particle:
    def __init__(self):
        self.position = random_timetable()
        self.best_position = self.position
        self.best_score = fitness(self.position)

    def update(self, global_best):
        new_position = {}
        for sec in sections:
            new_entries = []
            for i, (sub, slot, fac) in enumerate(self.position[sec]):
                # With probability move towards personal/global best
                if random.random() < 0.5:
                    slot = self.best_position[sec][i][1]
                if random.random() < 0.5:
                    slot = global_best[sec][i][1]
                new_entries.append((sub, slot, fac))
            new_position[sec] = new_entries

        new_position = repair(new_position)
        new_score = fitness(new_position)

        # Update personal best
        if new_score < self.best_score:
            self.best_position = new_position
            self.best_score = new_score

        self.position = new_position


def pso(num_particles=20, iterations=200):
    swarm = [Particle() for _ in range(num_particles)]
    global_best = min(swarm, key=lambda p: p.best_score).best_position
    global_best_score = fitness(global_best)

    for it in range(iterations):
        for particle in swarm:
            particle.update(global_best)
            if particle.best_score < global_best_score:
                global_best = particle.best_position
                global_best_score = particle.best_score

        print(f"Iteration {it+1}, Best Score = {global_best_score}")
        if global_best_score == 0:
            break

    return global_best, global_best_score


# ==============================
# Run PSO
# ==============================
best_timetable, best_score = pso()

# ==============================
# Display Timetable
# ==============================
for sec in sections:
    print(f"\nTimetable for {sec}:")
    for sub, slot, fac in best_timetable[sec]:
        day = days[slot // slots_per_day]
        hour = slot % slots_per_day + 1
        print(f"{day} Slot-{hour}: {sub} (Faculty {fac})")

print("\nFinal Best Score:", best_score)


Iteration 1, Best Score = 288
Iteration 2, Best Score = 286
Iteration 3, Best Score = 284
Iteration 4, Best Score = 284
Iteration 5, Best Score = 284
Iteration 6, Best Score = 284
Iteration 7, Best Score = 284
Iteration 8, Best Score = 280
Iteration 9, Best Score = 280
Iteration 10, Best Score = 280
Iteration 11, Best Score = 280
Iteration 12, Best Score = 280
Iteration 13, Best Score = 280
Iteration 14, Best Score = 280
Iteration 15, Best Score = 280
Iteration 16, Best Score = 280
Iteration 17, Best Score = 280
Iteration 18, Best Score = 280
Iteration 19, Best Score = 280
Iteration 20, Best Score = 280
Iteration 21, Best Score = 280
Iteration 22, Best Score = 280
Iteration 23, Best Score = 280
Iteration 24, Best Score = 280
Iteration 25, Best Score = 280
Iteration 26, Best Score = 280
Iteration 27, Best Score = 280
Iteration 28, Best Score = 280
Iteration 29, Best Score = 280
Iteration 30, Best Score = 280
Iteration 31, Best Score = 280
Iteration 32, Best Score = 280
Iteration 33, Bes

In [6]:
import random

# ==============================
# Problem Setup
# ==============================
days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
slots_per_day = 6   # 6 slots per day
total_slots = len(days) * slots_per_day  # 30 slots per week

sections = [f"S{i+1}" for i in range(22)]   # 22 sections
faculties = [chr(65+i) for i in range(11)]  # Faculty A-K

# Subjects with credits
subjects = {
    "CN": 4, "AI": 4, "PE": 3, "OOM": 3,
    "DEV": 3, "EVS": 1, "CC": 1, "BIS": 1,
    "MiniProject": 2
}

# Assign faculties to subjects (cyclic)
subject_faculty_map = {}
i = 0
for sub in subjects:
    subject_faculty_map[sub] = faculties[i % len(faculties)]
    i += 1

# ==============================
# Helper Functions
# ==============================
def random_timetable():
    timetable = {}
    for sec in sections:
        timetable[sec] = []
        for sub, hours in subjects.items():
            chosen_slots = random.sample(range(total_slots), hours)
            for slot in chosen_slots:
                timetable[sec].append((sub, slot, subject_faculty_map[sub]))
    return timetable

def fitness(timetable):
    conflicts = 0
    faculty_schedule = {fac: {} for fac in faculties}  # slot -> count

    for sec in sections:
        sec_slots = {}
        for sub, slot, fac in timetable[sec]:
            # Section-level conflicts
            if slot in sec_slots:
                conflicts += 1
            else:
                sec_slots[slot] = sub

            # Faculty-level conflicts: only count if faculty teaches >2 sections simultaneously
            if slot in faculty_schedule[fac]:
                faculty_schedule[fac][slot] += 1
                if faculty_schedule[fac][slot] > 2:
                    conflicts += 1
            else:
                faculty_schedule[fac][slot] = 1

    return conflicts

def repair(timetable):
    used_slots = {sec: set() for sec in sections}
    faculty_slots = {fac: {} for fac in faculties}

    for sec in sections:
        new_entries = []
        for sub, slot, fac in timetable[sec]:
            tries = 0
            while (slot in used_slots[sec]) or (faculty_slots.get(fac, {}).get(slot, 0) >= 2):
                slot = random.randint(0, total_slots - 1)
                tries += 1
                if tries > 100:
                    break
            new_entries.append((sub, slot, fac))
            used_slots[sec].add(slot)
            if fac not in faculty_slots:
                faculty_slots[fac] = {}
            if slot not in faculty_slots[fac]:
                faculty_slots[fac][slot] = 0
            faculty_slots[fac][slot] += 1
        timetable[sec] = new_entries
    return timetable

# ==============================
# PSO Implementation
# ==============================
class Particle:
    def __init__(self):
        self.position = random_timetable()
        self.best_position = self.position
        self.best_score = fitness(self.position)

    def update(self, global_best):
        new_position = {}
        for sec in sections:
            new_entries = []
            for i, (sub, slot, fac) in enumerate(self.position[sec]):
                if random.random() < 0.5:
                    slot = self.best_position[sec][i][1]
                if random.random() < 0.5:
                    slot = global_best[sec][i][1]
                new_entries.append((sub, slot, fac))
            new_position[sec] = new_entries

        new_position = repair(new_position)
        new_score = fitness(new_position)

        if new_score < self.best_score:
            self.best_position = new_position
            self.best_score = new_score

        self.position = new_position

def pso(num_particles=30, iterations=300):
    swarm = [Particle() for _ in range(num_particles)]
    global_best = min(swarm, key=lambda p: p.best_score).best_position
    global_best_score = fitness(global_best)

    for it in range(iterations):
        for particle in swarm:
            particle.update(global_best)
            if particle.best_score < global_best_score:
                global_best = particle.best_position
                global_best_score = particle.best_score

        print(f"Iteration {it+1}, Best Score = {global_best_score}")
        if global_best_score == 0:
            break

    return global_best, global_best_score

# ==============================
# Run PSO
# ==============================
best_timetable, best_score = pso()

# ==============================
# Display Timetable
# ==============================
for sec in sections:
    print(f"\nTimetable for {sec}:")
    sec_schedule = sorted(best_timetable[sec], key=lambda x: x[1])
    for sub, slot, fac in sec_schedule:
        day = days[slot // slots_per_day]
        hour = slot % slots_per_day + 1
        print(f"{day} Slot-{hour}: {sub} (Faculty {fac})")

print("\nFinal Best Score:", best_score)


Iteration 1, Best Score = 83
Iteration 2, Best Score = 81
Iteration 3, Best Score = 81
Iteration 4, Best Score = 81
Iteration 5, Best Score = 81
Iteration 6, Best Score = 81
Iteration 7, Best Score = 81
Iteration 8, Best Score = 81
Iteration 9, Best Score = 80
Iteration 10, Best Score = 80
Iteration 11, Best Score = 80
Iteration 12, Best Score = 80
Iteration 13, Best Score = 80
Iteration 14, Best Score = 79
Iteration 15, Best Score = 79
Iteration 16, Best Score = 79
Iteration 17, Best Score = 79
Iteration 18, Best Score = 79
Iteration 19, Best Score = 79
Iteration 20, Best Score = 79
Iteration 21, Best Score = 79
Iteration 22, Best Score = 79
Iteration 23, Best Score = 79
Iteration 24, Best Score = 79
Iteration 25, Best Score = 79
Iteration 26, Best Score = 79
Iteration 27, Best Score = 79
Iteration 28, Best Score = 79
Iteration 29, Best Score = 79
Iteration 30, Best Score = 79
Iteration 31, Best Score = 79
Iteration 32, Best Score = 79
Iteration 33, Best Score = 79
Iteration 34, Best 

In [7]:
import random

# ==============================
# Problem Setup
# ==============================
days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
slots_per_day = 6   # 6 slots per day
total_slots = len(days) * slots_per_day  # 30 slots per week

sections = [f"S{i+1}" for i in range(8)]   # 22 sections
faculties = [chr(65+i) for i in range(11)]  # Faculty A-K

# Subjects with credits
subjects = {
    "CN": 4, "AI": 4, "PE": 3, "OOM": 3,
    "DEV": 3, "EVS": 1, "CC": 1, "BIS": 1,
    "MiniProject": 2
}

# Assign faculties to subjects (cyclic)
subject_faculty_map = {}
i = 0
for sub in subjects:
    subject_faculty_map[sub] = faculties[i % len(faculties)]
    i += 1

# ==============================
# Helper Functions
# ==============================
def random_timetable():
    timetable = {}
    for sec in sections:
        timetable[sec] = []
        for sub, hours in subjects.items():
            chosen_slots = random.sample(range(total_slots), hours)
            for slot in chosen_slots:
                timetable[sec].append((sub, slot, subject_faculty_map[sub]))
    return timetable

def fitness(timetable):
    conflicts = 0
    faculty_schedule = {fac: {} for fac in faculties}  # slot -> count

    for sec in sections:
        sec_slots = {}
        for sub, slot, fac in timetable[sec]:
            # Section-level conflicts
            if slot in sec_slots:
                conflicts += 1
            else:
                sec_slots[slot] = sub

            # Faculty-level conflicts: only count if faculty teaches >2 sections simultaneously
            if slot in faculty_schedule[fac]:
                faculty_schedule[fac][slot] += 1
                if faculty_schedule[fac][slot] > 2:
                    conflicts += 1
            else:
                faculty_schedule[fac][slot] = 1

    return conflicts

def repair(timetable):
    used_slots = {sec: set() for sec in sections}
    faculty_slots = {fac: {} for fac in faculties}

    for sec in sections:
        new_entries = []
        for sub, slot, fac in timetable[sec]:
            tries = 0
            while (slot in used_slots[sec]) or (faculty_slots.get(fac, {}).get(slot, 0) >= 2):
                slot = random.randint(0, total_slots - 1)
                tries += 1
                if tries > 100:
                    break
            new_entries.append((sub, slot, fac))
            used_slots[sec].add(slot)
            if fac not in faculty_slots:
                faculty_slots[fac] = {}
            if slot not in faculty_slots[fac]:
                faculty_slots[fac][slot] = 0
            faculty_slots[fac][slot] += 1
        timetable[sec] = new_entries
    return timetable

# ==============================
# PSO Implementation
# ==============================
class Particle:
    def __init__(self):
        self.position = random_timetable()
        self.best_position = self.position
        self.best_score = fitness(self.position)

    def update(self, global_best):
        new_position = {}
        for sec in sections:
            new_entries = []
            for i, (sub, slot, fac) in enumerate(self.position[sec]):
                if random.random() < 0.5:
                    slot = self.best_position[sec][i][1]
                if random.random() < 0.5:
                    slot = global_best[sec][i][1]
                new_entries.append((sub, slot, fac))
            new_position[sec] = new_entries

        new_position = repair(new_position)
        new_score = fitness(new_position)

        if new_score < self.best_score:
            self.best_position = new_position
            self.best_score = new_score

        self.position = new_position

def pso(num_particles=30, iterations=300):
    swarm = [Particle() for _ in range(num_particles)]
    global_best = min(swarm, key=lambda p: p.best_score).best_position
    global_best_score = fitness(global_best)

    for it in range(iterations):
        for particle in swarm:
            particle.update(global_best)
            if particle.best_score < global_best_score:
                global_best = particle.best_position
                global_best_score = particle.best_score

        print(f"Iteration {it+1}, Best Score = {global_best_score}")
        if global_best_score == 0:
            break

    return global_best, global_best_score

# ==============================
# Run PSO
# ==============================
best_timetable, best_score = pso()

# ==============================
# Display Timetable
# ==============================
for sec in sections:
    print(f"\nTimetable for {sec}:")
    sec_schedule = sorted(best_timetable[sec], key=lambda x: x[1])
    for sub, slot, fac in sec_schedule:
        day = days[slot // slots_per_day]
        hour = slot % slots_per_day + 1
        print(f"{day} Slot-{hour}: {sub} (Faculty {fac})")

print("\nFinal Best Score:", best_score)


Iteration 1, Best Score = 0

Timetable for S1:
Mon Slot-1: CN (Faculty A)
Mon Slot-2: CC (Faculty G)
Mon Slot-3: CN (Faculty A)
Mon Slot-5: DEV (Faculty E)
Tue Slot-1: OOM (Faculty D)
Tue Slot-2: DEV (Faculty E)
Tue Slot-3: AI (Faculty B)
Tue Slot-4: AI (Faculty B)
Tue Slot-5: MiniProject (Faculty I)
Tue Slot-6: CN (Faculty A)
Wed Slot-1: PE (Faculty C)
Wed Slot-3: AI (Faculty B)
Wed Slot-4: BIS (Faculty H)
Wed Slot-6: EVS (Faculty F)
Thu Slot-1: PE (Faculty C)
Thu Slot-2: AI (Faculty B)
Thu Slot-3: DEV (Faculty E)
Thu Slot-6: PE (Faculty C)
Fri Slot-1: OOM (Faculty D)
Fri Slot-2: MiniProject (Faculty I)
Fri Slot-3: CN (Faculty A)
Fri Slot-6: OOM (Faculty D)

Timetable for S2:
Mon Slot-1: DEV (Faculty E)
Mon Slot-4: OOM (Faculty D)
Mon Slot-5: AI (Faculty B)
Mon Slot-6: MiniProject (Faculty I)
Tue Slot-2: OOM (Faculty D)
Tue Slot-3: AI (Faculty B)
Tue Slot-4: EVS (Faculty F)
Tue Slot-5: AI (Faculty B)
Tue Slot-6: PE (Faculty C)
Wed Slot-1: BIS (Faculty H)
Wed Slot-2: AI (Faculty B)
Wed